In [6]:
import pandas as pd
import numpy as np
import random

# 1. Veriyi pkl dosyasından yükle
file_path = 'hazirlanmis_etkilesim.pkl'
data = pd.read_pickle(file_path)

# İlk birkaç satırı görüntüle
print("Veri Setindeki İlk 5 Satır:")
print(data.head())

print("\n")


Veri Setindeki İlk 5 Satır:
    uye_id  urun_id           eklenme_tarihi  puan   event_type
0        0    72176  2024-08-05 05:43:02.000     1         view
1        0    67714  2024-08-05 05:45:51.000     5     purchase
2        0    66060  2024-08-05 05:53:56.000     4  add_to_cart
3  1659156    60388  2024-08-05 06:01:07.000     1         view
4        0    41669  2024-08-05 06:04:45.000     5     purchase




In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Birlikte alınan ürünleri belirlemek için kullanıcı üzerinden grup oluşturma
data['birlikte_urun_id'] = data.groupby('uye_id')['urun_id'].shift(-1)

# Sadece geçerli eşleşmelerin olduğu satırları tut
data = data.dropna()
# Kendisiyle ilişkili ürünleri çıkar
data = data[data['urun_id'] != data['birlikte_urun_id']]
data['birlikte_urun_id'] = data['birlikte_urun_id'].astype(int)

# Özellikler ve hedef
X = data[['urun_id']]  # Sadece ürün ID'yi kullanıyoruz
y = data['birlikte_urun_id']  # Tahmin edilmesi gereken birlikte ürün ID

# Eğitim ve test verisine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Modeli
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Örnek bir ürün ID ile tahmin yapma
example_product_id = [[72176]]  # Örnek ürün ID
predicted_product_id = rf.predict(example_product_id)

print(f"Ürün ID {example_product_id[0][0]} için önerilen ürün ID: {predicted_product_id[0]}")

Ürün ID 72176 için önerilen ürün ID: 73914


C:\Users\ckasikci\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [14]:
def calculate_cc(recommendations, product_pool):
    # Önerilen benzersiz ürünler
    recommended_items = set(recommendations)
    
    # CC Hesaplama
    coverage = len(recommended_items) / len(product_pool)
    return coverage

# Veri setindeki tüm ürünler
product_pool = data['urun_id'].unique()

# Test seti için tahminler
recommendations = rf.predict(X_test)

# Catalog Coverage hesapla
cc_score = calculate_cc(recommendations, product_pool)
print(f"Catalog Coverage (CC): {cc_score:.4f}")



Catalog Coverage (CC): 0.2395


In [15]:
import pandas as pd

# y_test'i one-hot encode yap
y_test_encoded = pd.get_dummies(y_test).reindex(columns=rf.classes_, fill_value=0)
print(f"y_test_encoded formatı: {type(y_test_encoded)}, boyut: {y_test_encoded.shape}")



y_test_encoded formatı: <class 'pandas.core.frame.DataFrame'>, boyut: (29930, 3130)


In [16]:
from sklearn.metrics import accuracy_score

def calculate_mps(y_true, y_pred):

    # Doğru tahmin oranını hesapla
    mps = accuracy_score(y_true, y_pred)
    return mps
# Test veri seti üzerinde tahmin yap
y_pred = rf.predict(X_test)

# MPS hesapla
mps_score = calculate_mps(y_test, y_pred)
print(f"Mean Predictive Score (MPS): {mps_score:.4f}")


Mean Predictive Score (MPS): 0.3360


In [17]:
import numpy as np

def calculate_mns(data, model, X_test):

    # 1. Ürün popülerlik skorlarını hesapla
    product_popularity = data['urun_id'].value_counts(normalize=True).to_dict()

    # 2. Modelin önerdiği ürünleri al
    recommendations = model.predict(X_test)

    # 3. MNS hesapla
    novelty_scores = [
        -np.log(product_popularity.get(product_id, 1e-10))  # Popülerlik sıfırsa küçük bir değer kullan
        for product_id in recommendations
    ]
    
    return np.mean(novelty_scores)

# MNS hesapla
mns_score = calculate_mns(data, rf, X_test)
print(f"Mean Novelty Score (MNS): {mns_score:.4f}")


Mean Novelty Score (MNS): 3.8969


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def calculate_mds(data, model, X_test, sample_size=100):
    df = data.copy()

    # Gerekli sütunları kontrol et
    if 'eklenme_tarihi' not in df.columns or 'puan' not in df.columns:
        print("Hata: 'eklenme_tarihi' veya 'puan' sütunu eksik.")
        return 0.0

    # Tarihi dönüştür
    df['eklenme_tarihi'] = pd.to_datetime(df['eklenme_tarihi'], errors='coerce')
    df = df.dropna(subset=['eklenme_tarihi', 'puan'])

    # Epoch formatına çevir
    df['eklenme_tarihi'] = df['eklenme_tarihi'].astype('int64') / 10**9

    # Özellik matrisini hazırla
    product_features = df.set_index('urun_id')[['puan', 'eklenme_tarihi']]

    # Modelin tahmin ettiği ürünleri al
    recommendations = model.predict(X_test)

    # Benzersiz ürünleri al
    unique_recommendations = np.unique(recommendations)
    if len(unique_recommendations) > sample_size:
        unique_recommendations = np.random.choice(unique_recommendations, sample_size, replace=False)

    if len(unique_recommendations) <= 1:
        return 0.0

    try:
        features = product_features.loc[unique_recommendations].dropna().values
        similarity_matrix = cosine_similarity(features)
        mean_sim = np.mean(similarity_matrix[np.triu_indices(len(features), k=1)])
        return round(1 - mean_sim, 4)
    except Exception as e:
        print("Hata oluştu:", e)
        return 0.0
mds_score = calculate_mds(data, model=rf, X_test=X_test)
print(f"Mean Diversity Score (MDS): {mds_score:.4f}")


Mean Diversity Score (MDS): 0.0000


In [19]:
def calculate_map(model, X_test, y_test, k=1):
    correct_predictions = 0
    total_users = len(X_test)

    for i in range(total_users):
        input_item = X_test.iloc[[i]]  # Sütun adları korunur
        true_label = y_test.iloc[i]

        predicted = model.predict(input_item)

        if true_label in predicted[:k]:
            correct_predictions += 1

    map_score = correct_predictions / total_users
    return round(map_score, 4)

map_score = calculate_map(model=rf, X_test=X_test, y_test=y_test, k=1)
print(f"Mean Average Precision (MAP): {map_score:.4f}")


Mean Average Precision (MAP): 0.3360


In [20]:
import pickle

# Tüm metrikleri tek bir sözlükte topla
random_forest_results = {
    "MAP": map_score,
    "CC": cc_score,
    "MNS": mns_score,
    "MDS": mds_score,
    "MPS": mps_score
}

# Tek bir dosyaya kaydet
with open("random_forest_sonuclari.pkl", "wb") as f:
    pickle.dump(random_forest_results, f)
